In [16]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
import torch
from torch import nn
from torch.optim import lr_scheduler 
from torch.utils.data import Dataset, DataLoader

#setting basic environment
torch.manual_seed(100)

#if we have a gpu then run network on that, if not use cpu which is much slower
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#" Varying learning rate between 0.0001 and 0.01 is considered optimal in most of the cases" - https://www.kdnuggets.com/2022/12/tuning-adam-optimizer-parameters-pytorch.html
lr = 0.001
batch_size = 25
num_epochs = 500
loss_fn = nn.BCELoss()
scaler = MinMaxScaler()

class CustomDataset(Dataset):
    def __init__(self, dataframe):
       self.dataframe = dataframe.apply(pd.to_numeric, errors='coerce')

    def __getitem__(self, index):
       row = self.dataframe.iloc[index].to_numpy()
       features = row[1:26]
       label = row[26]
       features_tensor = torch.tensor(features, dtype=torch.float32)
       label_tensor = torch.tensor(label, dtype=torch.float32)
       return features_tensor, label_tensor
    
    def __len__(self):
       return len(self.dataframe)


csv_training_data = pd.read_csv('training data.csv')
train_data = csv_training_data.to_numpy()

X_train = np.array(train_data[556:,2:27],dtype=float)
X_train_normalized = scaler.fit(X_train)

X_test = np.array(train_data[1300:,2:27],dtype=float)
X_test_normalized = scaler.fit(X_test)

Y_train = np.array(train_data[556:,27].transpose(), dtype=float)
Y_test = np.array(train_data[1300:,27].transpose(), dtype=float)

#solver default Adam, activation default relu, shuffle default true
clf = MLPClassifier(batch_size=100, learning_rate_init=0.01, hidden_layer_sizes=(50,100,50), 
                    max_iter=500)
clf = clf.fit(X_train_normalized, Y_train)
scores = sklearn.model_selection.cross_val_score(clf,X_train_normalized,Y_train,cv=5)
print(scores)

TypeError: float() argument must be a string or a real number, not 'MinMaxScaler'